In [ ]:
# default_exp mlb_api

# Parse MLB API

> details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#exporti

import statsapi as mlb
from typing import List

In [ ]:
import datetime as dt
from pydantic import validate_arguments

@validate_arguments
def get_days_games(date:dt.date=dt.date.today(),sportId:int=1):
    response = mlb.get(
        'schedule',{
            'date':date.strftime('%Y-%m-%d'),
            'sportId':1
        }
    )
    assert len(response['dates'])==1
    return response['dates'][0]['games']

In [ ]:
len(get_days_games(date='1994-06-11',sportId='1'))

In [ ]:
#exporti 

def get_season_game_pks(season: int) -> List[int]:
    """
    Returns a list of gamePks for a given season
    
    :param season: int for the year of the season 
    
    :returns list of gamePks as integers
    """
    season = mlb.get('season',{'sportId':1,'seasonId':season})['seasons'][0]
    start = season['seasonStartDate']
    end = season['seasonEndDate']
    schedule = mlb.get(
        'schedule',
        {'startDate':start,
         'endDate':end,
         'sportId':1}
    )

    pks = list(reversed(
        [game['gamePk'] 
         for dates in schedule['dates']
         for game in dates['games'] 
        ]
    ))
    pks = list(reversed(pks))
    return pks

In [ ]:
show_doc(get_season_game_pks)

In [ ]:
gamePks = get_season_game_pks(2021)[:5]
assert (len(gamePks))==5
for pk in gamePks:
    assert type(pk)==int

In [ ]:
game_details = mlb.get('schedule',{'sportId':1,'date':'2021-04-01'})['dates'][0]['games'][0]

In [ ]:
#exporti 

def add_playFKs(dictionary,play,game):
    """
    For the nested events within a play, add the foreign keys needed to trace the event 
    back to it's game/play
    """
    for fk in ['atBatIndex','playEndTime']:
        dictionary[fk]=play[fk]
        dictionary['gamePk']=game['gamePk']
        
def parse(input_dict,nested_lists=None):
    """recursively parse a dictionary with nested lists into a flat dictionary"""
    out={}
    if dict not in [type(x) for x in input_dict.values()]:
        return input_dict
    else:
        for k,v in input_dict.items():
            if type(v) in [str,int,float,bool]:
                out[k]=input_dict[k]
            elif type(v)==dict:
                nested_dict = v
                # if nested dict has an 'id', use it as a foreign key
                # exepct in the cause of the 'game' table which has a pk field
                if 'id' in nested_dict.keys() and 'pk' not in nested_dict.keys():
                    out[f"{k}_id"]=nested_dict['id']
                else:
                    for key,value in nested_dict.items():
                        out[f"{k}_{key}"]=value
            elif type(v)==list:
                if nested_lists != None:
                    nested_lists.append({k:v})
        return parse(out,nested_lists)

In [ ]:
show_doc(add_playFKs)

In [ ]:
show_doc(parse)

In [ ]:
#export 

class Game():
    """Reference obj for a the result of an MLB API call."""
    
    def __init__(self,gamePk):
        """Call the mlb API and parse results into flat dictionaries for DB insert."""
        # call the api 
        game = mlb.get('game',{'gamePk':gamePk})
        self._result = game
        
        # players
        self.players = []
        players = game['gameData'].pop('players')
        for playerId in players.keys():
            self.players.append(parse(players[playerId]))
        
        # team / player stats
        #self.teams = []
        self.game_players = []
        self.team_stats = []
        self.team_records = []
        for home_away in ['away','home']:
            team = game['liveData']['boxscore']['teams'][home_away]
            players = team.pop('players')
            for playerId in players.keys():
                player = players[playerId]
                # add foreign keys to player / game records
                player['team_id']=team['team']['id']
                player['gamePk']=game['gamePk']
                self.game_players.append(parse(player))
            team_stats = team.pop('teamStats')
            team_stats['team_id']=team['team']['id']
            team_stats['gamePk']=game['gamePk']
            self.team_stats.append(parse(team_stats))
            #self.teams.append(parse(team))
        
        # game
        gm = parse(game['gameData'])
        #gm.update(game['gameData']['game'])
        setattr(self,'game',gm)
        
        # teams / team records 
        self.teams = []
        self.team_records = []
        teams = game['gameData']['teams']
        for home_away in ['home','away']:
            team = teams[home_away]
            team_record = parse(team.pop('record'))
            team_record['teamId']=team['id']
            team_record['gamePk']=game['gamePk']
            self.team_records.append(team_record)
            self.teams.append(parse(team))
        
        # venue
        self.venue = parse(game['gameData']['venue'])
        
        # plays and play events 
        parsed_plays = []
        game_play_events = []
        matchups = []
        self.pitches = []
        self.pitchData = []
        self.hitData = []
        self.actions = []
        self.movements = []
        self.credits = []
        for play in game['liveData']['plays']['allPlays']:
            matchup = play.pop('matchup')
            add_playFKs(matchup,play,game)
            matchups.append(matchup)
            
            nested_play_details = []
            play_events = []
            
            parsed_plays.append(parse(play,nested_lists=nested_play_details))
            
            
            for element in nested_play_details:
                for value in element.values():
                    if type(value)==list and len(value)>0:
                        if type(value[0])==dict:
                            for x in value:
                                play_event = parse(x,nested_play_details)
                                # add FKs to trace back to the play
                                add_playFKs(play_event,play,game)
                                if play_event.get('isPitch'):
                                    pitchFKs = ['gamePk','atBatIndex','playEndTime','index']
                                    
                                    pitchData_keys = [x for x in play_event.keys() if x.startswith('pitchData')]
                                    if len(pitchData_keys)>0:
                                        pitchData = {}
                                        for _key in pitchFKs:
                                            pitchData[_key]=play_event[_key]
                                        for _key in pitchData_keys:
                                            pitchData[_key.replace('pitchData_','')]=play_event.pop(_key)
                                        self.pitchData.append(pitchData)
                                    
                                    hitData_keys = [x for x in play_event.keys() if x.startswith('hitData')]
                                    if len(hitData_keys)>0:
                                        hitData = {}
                                        for _key in pitchFKs:
                                            hitData[_key]=play_event[_key]
                                        for _key in hitData_keys:
                                            hitData[_key.replace('hitData_','')]=play_event.pop(_key)
                                        self.hitData.append(hitData)
                                                
                                    self.pitches.append(play_event)
                                if play_event.get('type')=='action':
                                    self.actions.append(play_event)
                                if 'movement_isOut' in list(play_event.keys()):
                                    self.movements.append(play_event)
                                if 'credit' in list(play_event.keys()):
                                    self.credits.append(play_event)
                                play_events.append(play_event)
                                
            game_play_events.append(play_events)
            
        for play in parsed_plays:
            play['gamePk']=game['gamePk']
        
        setattr(self,'plays',parsed_plays)
        #setattr(self,'play_events',game_play_events)
        
        # dealing with matchups
        parsed_matchups = []
        game_matchup_stats = []
        for matchup in matchups:
            # pop out hot cold stats
            # these are such a pain to parse I might just calculate them myself
            for stats in ['batterHotColdZoneStats','pitcherHotColdZoneStats']:
                if stats in matchup.keys():
                    game_matchup_stats.append(matchup.pop(stats))
            parsed_matchups.append(parse(matchup))
                            
        setattr(self,'matchups',parsed_matchups)
    
    def __repr__(self):
        return self.game['game_id']

In [ ]:
show_doc(Game)

In [ ]:
game = Game(gamePks[0])

In [ ]:
game.game

In [ ]:
game.movements[:2]

In [ ]:
game.game_players[0]

In [ ]:
game.teams